In [5]:
import numpy as np

def soc_distance(socs, lower_bound=20, upper_bound=30):
    """
    Calcula la distancia del SoC a los límites críticos de 20% y 30%.
    """
    distances = np.where(socs < lower_bound, lower_bound - socs, 
                         np.where(socs > upper_bound, socs - upper_bound, 0))
    return distances

def analyze_time_in_soc_ranges_ext(socs, dt, ranges=np.arange(0, 110, 10)):
    """
    Analiza el tiempo pasado en distintos rangos de SoC.

    Args:
        socs (np.array): Array con los valores de SoC a lo largo del tiempo.
        dt (float): Intervalo de tiempo entre mediciones de SoC.
        ranges (np.array, opcional): Rangos de SoC para el análisis. Por defecto va de 0 a 100 en pasos de 10.

    Returns:
        dict: Tiempo pasado en cada rango de SoC.
    """
    time_in_ranges = {f"{ranges[i]}%-{ranges[i+1]}%": 0 for i in range(len(ranges) - 1)}

    for soc in socs:
        index = np.digitize(soc, ranges) - 1
        index = min(index, len(ranges) - 2)
        range_key = f"{ranges[index]}%-{ranges[index+1]}%"
        time_in_ranges[range_key] += dt

    return time_in_ranges


def evaluate_performance_ext(time_in_ranges, distances, weights=None):
    """
    Evalúa el rendimiento del sistema basado en el tiempo en rangos de SoC y las distancias a los límites críticos.
    """
    if weights is None:
        weights = {"20%-30%": 2, "0%-20%": 5}

    performance_score = sum(time * weights.get(range_key, 1) for range_key, time in time_in_ranges.items())
    penalty = np.sum(distances)
    return performance_score - penalty
# ejemplo
times = np.arange(0, 97, 1)  # De 0 a 100 minutos
socs = np.linspace(95, 5, len(times))  # SoC decreciente de 95% a 5%

distances = soc_distance(socs)
time_in_ranges = analyze_time_in_soc_ranges_ext(socs, 1) #dt=1
performance_score = evaluate_performance_ext(time_in_ranges, distances)

print("Time in each SoC range:", time_in_ranges)
print("Performance score:", performance_score)
print(distances)


Time in each SoC range: {'0%-10%': 6, '10%-20%': 10, '20%-30%': 11, '30%-40%': 11, '40%-50%': 10, '50%-60%': 11, '60%-70%': 11, '70%-80%': 10, '80%-90%': 11, '90%-100%': 6}
Performance score: -2305.4375
[65.     64.0625 63.125  62.1875 61.25   60.3125 59.375  58.4375 57.5
 56.5625 55.625  54.6875 53.75   52.8125 51.875  50.9375 50.     49.0625
 48.125  47.1875 46.25   45.3125 44.375  43.4375 42.5    41.5625 40.625
 39.6875 38.75   37.8125 36.875  35.9375 35.     34.0625 33.125  32.1875
 31.25   30.3125 29.375  28.4375 27.5    26.5625 25.625  24.6875 23.75
 22.8125 21.875  20.9375 20.     19.0625 18.125  17.1875 16.25   15.3125
 14.375  13.4375 12.5    11.5625 10.625   9.6875  8.75    7.8125  6.875
  5.9375  5.      4.0625  3.125   2.1875  1.25    0.3125  0.      0.
  0.      0.      0.      0.      0.      0.      0.      0.      0.
  0.9375  1.875   2.8125  3.75    4.6875  5.625   6.5625  7.5     8.4375
  9.375  10.3125 11.25   12.1875 13.125  14.0625 15.    ]
